In [1]:
import datetime as dt
from utility import (
    read_sql,
    fd_alive_funds,
    fd_basicinfo,
    fd_typeclass,
    fd_hshkiport,
    fd_assetportfolio,
    fd_qtfdnav,
    nearest_report_date
)
import pandas as pd
from PyFin.api import makeSchedule
from PyFin.api import BizDayConventions

# 1. 基金分类
-----------------------

In [2]:
# 获取回溯的报告日

today = dt.datetime.today()
today = dt.date(2015, 1, 30)
date_3yrs_ago = today.replace(year=today.year - 3)
report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
current_date = today.strftime("%Y%m%d")

In [3]:
# 获取所有目标混合型基金（清算结束日期晚于当前日）

security_ids = fd_alive_funds(report_dates_begin, 202)

In [4]:
# 获取相关组合情况

basic_info = fd_basicinfo(security_ids, current_date)
type_info = fd_typeclass(security_ids, current_date)
hkport_info = fd_hshkiport(security_ids, report_dates_begin, current_date)
hkport_info = hkport_info[hkport_info.INDUSTRYNAME == "合计"].groupby(["SECURITYID"], as_index=False)[["ACCNETMKTCAP"]].mean()
asset_port = fd_assetportfolio(security_ids, report_dates_begin, current_date).groupby("SECURITYID", as_index=False)[["EQUITYINVERTO"]].mean()

In [5]:
df = pd.merge(basic_info, type_info, on="SECURITYID")
df = pd.merge(df, asset_port, on="SECURITYID", how="left")
df = pd.merge(df, hkport_info, on="SECURITYID", how="left")
if "ACCNETMKTCAP" in df:
    df["ACCNETMKTCAP"] = df["ACCNETMKTCAP"].fillna(0)
else:
    df["ACCNETMKTCAP"] = 0.0

In [6]:
# 分类
flag = df["ACCNETMKTCAP"] / df["EQUITYINVERTO"] > 0.5
df.loc[flag, "混合型（子类）"] = "港股通混合型基金"
df.loc[(~flag) & (df["EQUITYINVERTO"] >= 70), "混合型（子类）"] = "高权益仓位混合型基金"
df.loc[((~flag) & (df["EQUITYINVERTO"] >= 50) & (df["EQUITYINVERTO"] < 70)), "混合型（子类）"] = "中高权益仓位混合型基金"
df.loc[((~flag) & (df["EQUITYINVERTO"] >= 30) & (df["EQUITYINVERTO"] < 50)), "混合型（子类）"] = "中权益仓位混合型基金"
df.loc[((~flag) & (df["EQUITYINVERTO"] >= 0) & (df["EQUITYINVERTO"] < 30)), "混合型（子类）"] = "低权益仓位混合型基金"

In [7]:
df

,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,EQUITYINVERTO,ACCNETMKTCAP,混合型（子类）
0,1030000006,华夏成长证券投资基金,None,000001,证券投资基金,成长型,2,混合基金,2.1,偏股型基金,2.1.2,偏股型基金(股票上限80%),65.283333,0.0,中高权益仓位混合型基金
1,1030000007,华夏大盘精选证券投资基金,华夏大盘精选混合,000011,证券投资基金,增值型,2,混合基金,2.1,偏股型基金,2.1.1,偏股型基金(股票上限95%),82.620833,0.0,高权益仓位混合型基金
2,1030000016,华夏回报证券投资基金,华夏回报混合,002001,证券投资基金,收益型,2,混合基金,2.9,特定策略混合型基金,2.9.1,特定策略混合型基金,56.847500,0.0,中高权益仓位混合型基金
3,1030000016,华夏回报证券投资基金,华夏回报混合,002001,证券投资基金,收益型,2,混合基金,2.6,绝对收益目标基金,2.6.3,灵活策略基金（A类）,56.847500,0.0,中高权益仓位混合型基金
4,1030000017,华夏红利混合型证券投资基金,None,002011,证券投资基金,分红型,2,混合基金,2.2,灵活配置型基金,2.2.3,灵活配置型基金(股票上限95%)（A类）,83.277500,0.0,高权益仓位混合型基金
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,1030000302,华泰柏瑞积极成长混合型证券投资基金,None,460002,证券投资基金,积极成长型,2,混合基金,2.1,偏股型基金,2.1.1,偏股型基金(股票上限95%),84.670000,0.0,高权益仓位混合型基金
176,1030000383,汇丰晋信动态策略混合型证券投资基金,汇丰晋信动态策略混合,540003,证券投资基金,增值型,2,混合基金,2.1,偏股型基金,2.1.1,偏股型基金(股票上限95%),85.286667,0.0,高权益仓位混合型基金
177,1030000384,汇丰晋信2026生命周期证券投资基金,None,540004,证券投资基金,稳健成长型,2,混合基金,2.10,其他混合型基金,2.10.1,其他混合型基金,80.030833,0.0,高权益仓位混合型基金
178,1030000256,兴全可转债混合型证券投资基金,None,340001,证券投资基金,收益型,2,混合基金,2.4,偏债型基金,2.4.1,偏债型基金,21.677500,0.0,低权益仓位混合型基金


## 1.1 Put it all totether

In [12]:
def create_mutua_fund_info(trade_dt):
    trade_dt = dt.datetime.strptime(trade_dt, "%Y%m%d")
    date_3yrs_ago = trade_dt.replace(year=trade_dt.year - 3)
    report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
    current_date = trade_dt.strftime("%Y%m%d")
    
    security_ids = fd_alive_funds(report_dates_begin, 202)
    basic_info = fd_basicinfo(security_ids, current_date)
    type_info = fd_typeclass(security_ids, current_date)
    hkport_info = fd_hshkiport(security_ids, report_dates_begin, current_date)
    hkport_info = hkport_info[hkport_info.INDUSTRYNAME == "合计"].groupby(["SECURITYID"], as_index=False)[["ACCNETMKTCAP"]].mean()
    asset_port = fd_assetportfolio(security_ids, report_dates_begin, current_date).groupby("SECURITYID", as_index=False)[["EQUITYINVERTO"]].mean()

    df = pd.merge(basic_info, type_info, on="SECURITYID")
    df = pd.merge(df, asset_port, on="SECURITYID", how="left")
    df = pd.merge(df, hkport_info, on="SECURITYID", how="left").drop_duplicates()
    if "ACCNETMKTCAP" in df:
        df["ACCNETMKTCAP"] = df["ACCNETMKTCAP"].fillna(0)
    else:
        df["ACCNETMKTCAP"] = 0.0
    
    flag = df["ACCNETMKTCAP"] / df["EQUITYINVERTO"] > 0.5
    df.loc[flag, "混合型（子类）"] = "港股通混合型基金"
    df.loc[(~flag) & (df["EQUITYINVERTO"] >= 70), "混合型（子类）"] = "高权益仓位混合型基金"
    df.loc[((~flag) & (df["EQUITYINVERTO"] >= 50) & (df["EQUITYINVERTO"] < 70)), "混合型（子类）"] = "中高权益仓位混合型基金"
    df.loc[((~flag) & (df["EQUITYINVERTO"] >= 30) & (df["EQUITYINVERTO"] < 50)), "混合型（子类）"] = "中权益仓位混合型基金"
    df.loc[((~flag) & (df["EQUITYINVERTO"] >= 0) & (df["EQUITYINVERTO"] < 30)), "混合型（子类）"] = "低权益仓位混合型基金"
    
    return df

In [13]:
%%time

create_mutua_fund_info("20211111").groupby( "混合型（子类）").count()

Wall time: 10.7 s


,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,EQUITYINVERTO,ACCNETMKTCAP
混合型（子类）,,,,,,,,,,,,,,
中权益仓位混合型基金,139,139,102,139,139,139,139,139,139,139,139,139,139,139
中高权益仓位混合型基金,182,182,100,182,182,182,182,182,182,182,182,182,182,182
低权益仓位混合型基金,489,489,418,489,489,489,489,489,489,489,489,489,489,489
港股通混合型基金,17,17,6,17,17,17,17,17,17,17,17,17,17,17
高权益仓位混合型基金,1147,1147,405,1147,1147,1147,1147,1147,1147,1147,1147,1147,1147,1147


# 2. 基金指数
-----------------

In [16]:
# 我们只计算指数的每日收益
# 在确定指数的基准日之后，可以直接使用收益计算指数的值。
# 使用 TQ_FD_DERIVEDN 获取基金的净值情况

start_dt = "2015-01-31"
final_trade_dt = "2021-11-09"
nav_type = "ACCUNITNAV"

rebalance_dates = [d.strftime("%Y%m%d") for d in makeSchedule(start_dt, final_trade_dt, tenor="3M", calendar="china.sse", dateRule=BizDayConventions.ModifiedFollowing)]

In [17]:
dfs = []

for i, trade_dt in enumerate(rebalance_dates[1:]):
    pre_trade_dt = rebalance_dates[i]
    print(pre_trade_dt, trade_dt)
    fund_info = create_mutua_fund_info(trade_dt)
    pre_nav_info = fd_qtfdnav(fund_info.SECURITYID.tolist(), pre_trade_dt).rename(columns={nav_type: "PRE" + nav_type})
    nav_info = fd_qtfdnav(fund_info.SECURITYID.tolist(), trade_dt)

    total_df = pd.merge(fund_info, nav_info, on=["SECURITYID"])
    total_df = pd.merge(total_df, pre_nav_info, on=["SECURITYID"])
    total_df["chg."] = total_df[nav_type] / total_df["PRE" + nav_type] - 1.0
    res = total_df.groupby("混合型（子类）")[["chg."]].mean()
    dfs.append(res)

20150130 20150430
20150430 20150731
20150731 20151030
20151030 20160129
20160129 20160429
20160429 20160729
20160729 20161031
20161031 20170126
20170126 20170428
20170428 20170731
20170731 20171031
20171031 20180131
20180131 20180427
20180427 20180731
20180731 20181031
20181031 20190131
20190131 20190430
20190430 20190731
20190731 20191031
20191031 20200123
20200123 20200430
20200430 20200731
20200731 20201030
20201030 20210129
20210129 20210430
20210430 20210730
20210730 20211029
20211029 20211109


In [18]:
final_report = pd.concat(dfs, keys=rebalance_dates[1:]).reset_index()
final_report.pivot_table(index="level_0", columns="混合型（子类）", values="chg.").to_excel("020_混合型基金_bak.xlsx")

In [19]:
final_report.pivot_table(index="level_0", columns="混合型（子类）", values="chg.")

混合型（子类）,中权益仓位混合型基金,中高权益仓位混合型基金,低权益仓位混合型基金,港股通混合型基金,高权益仓位混合型基金
level_0,,,,,
20150430,0.066698,0.214053,0.043608,NaN,0.265505
20150731,-0.005351,-0.044305,-0.000181,NaN,-0.082734
20151030,0.002833,0.010425,0.006977,NaN,-0.005585
20160129,-0.014604,-0.077349,-0.003181,NaN,-0.107363
20160429,0.024989,0.048838,0.008376,NaN,0.053057
20160729,0.009016,0.024041,0.017015,NaN,0.022839
20161031,0.011226,0.016295,0.007070,NaN,0.014460
20170126,-0.011338,-0.017651,-0.005629,NaN,-0.032592
20170428,0.001123,0.021778,0.011195,NaN,0.018230
